## Feature Engineering

The steps in this section are
1. Missing values
2. Temporal variables
3. Categorical variables - Remove the less frequent ones
4. Standardise the values to the same range

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

In [30]:
housing_data = pd.read_csv('train.csv')
housing_data.shape

(1460, 81)

##### 1. Missing Values

Since Kaggle provides test and train data separately, we don't need to do the priliminary step of train_test_split to prevent data leakage

In [31]:
# First things first, let's convert the GarageYrBlt to int64

housing_data['GarageYrBlt'] = housing_data['GarageYrBlt'].astype('Int64')

In [32]:
# Let's get the columns with missing values

features_with_na = [feature for feature in housing_data.columns if housing_data[feature].isnull().sum()>1]

#Let's print the percentage of missing values in each of these columns
for feature in features_with_na:
    print('{} has {} % of missing values'.format(feature, (housing_data[feature].isnull().sum()/housing_data.shape[0])*100))

LotFrontage has 17.73972602739726 % of missing values
Alley has 93.76712328767123 % of missing values
MasVnrType has 0.547945205479452 % of missing values
MasVnrArea has 0.547945205479452 % of missing values
BsmtQual has 2.5342465753424657 % of missing values
BsmtCond has 2.5342465753424657 % of missing values
BsmtExposure has 2.6027397260273974 % of missing values
BsmtFinType1 has 2.5342465753424657 % of missing values
BsmtFinType2 has 2.6027397260273974 % of missing values
FireplaceQu has 47.26027397260274 % of missing values
GarageType has 5.5479452054794525 % of missing values
GarageYrBlt has 5.5479452054794525 % of missing values
GarageFinish has 5.5479452054794525 % of missing values
GarageQual has 5.5479452054794525 % of missing values
GarageCond has 5.5479452054794525 % of missing values
PoolQC has 99.52054794520548 % of missing values
Fence has 80.75342465753424 % of missing values
MiscFeature has 96.30136986301369 % of missing values


In [33]:
# AS we can see, there are 4 features with over 80% of values missing. It's safe to discard them
features_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature']
housing_data = housing_data.drop(features_to_drop, axis=1)
for feature in features_to_drop:
    features_with_na.remove(feature)

In [35]:
# Let's first deal with the missing values in the Categorical columns

cat_na_features = [feature for feature in features_with_na if housing_data[feature].dtype == 'O']

housing_data[cat_na_features].isnull().sum()

MasVnrType        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
FireplaceQu     690
GarageType       81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64

In [36]:
# Let's fill the missing values with 'Miss_val'

def replace_the_missing(housing_data, feature_list):
    house_data = housing_data.copy()
    house_data[feature_list] = house_data[feature_list].fillna('Miss_Val')
    return house_data

housing_data = replace_the_missing(housing_data, cat_na_features)
housing_data[cat_na_features].isnull().sum()

MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
dtype: int64

In [37]:
housing_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Miss_Val,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


Categorical columns with missing data has been filled with 'Miss_Val'

In [38]:
# Let's now go to the numerical side of the missing values
num_na_features = [feature for feature in features_with_na if feature not in cat_na_features]

print(num_na_features)
housing_data[num_na_features].isnull().sum()

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [39]:
# As can be seen from the exploratory analysis that these features have outliers.
# Hence, mean cannot be used to fill the missing values.
# Let's fill the missing values with the median of the columns.

for feature in num_na_features:
    # Calculate median
    median_val = housing_data[feature].median()
    
    # Create a new column to denote that there was a missing value (binary substitution) and then replace the current missing value with median
    housing_data[feature+'_NaN'] = np.where(housing_data[feature].isnull(), 1,0)
    housing_data[feature].fillna(median_val, inplace=True)
    
housing_data[num_na_features].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

##### 2. Temporal columns 

Back in EDA, we found 4 columns in the data that had values of Years. 

In [40]:
years_features = [feature for feature in housing_data.columns if ('Yr' in feature or 'Year' in feature) and 'NaN' not in feature]
years_features

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

Since the SalePrice was decreasing with increase in the YrSold, it is not the correct feature.
To make use of these years data, lets use the difference between the YrSold with other year columns.

In [41]:
for feature in years_features:
    if feature != 'YrSold':
        housing_data[feature+'_diff'] = housing_data['YrSold'] - housing_data[feature]

housing_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN,YearBuilt_diff,YearRemodAdd_diff,GarageYrBlt_diff
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Miss_Val,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500,0,0,0,5,5,5
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500,0,0,0,31,31,31
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500,0,0,0,7,6,7
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000,0,0,0,91,36,8
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000,0,0,0,8,8,8


##### 3. Handle Skewed data using log normal distribution


In [42]:
# From the Exploratorary analysis, we had found 5 columns with skewed data. 
# Let's get the log value to convert the skewed distribution to log normal distribution

skew_features = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']

for feature in skew_features:
    housing_data[feature] = np.log(housing_data[feature])

##### 4. Handle less frequent Categorical data

In [43]:
cat_features = [feature for feature in housing_data.columns if housing_data[feature].dtype == 'O']
cat_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [50]:
# If the categorical value is present less than 1% in a column, then that data isn't significant.
# Let's figure out the percentage of each values in categorical columns and replace the less frequent one with a common value across the columns

for feature in cat_features:
    temp = housing_data.groupby(feature)['LotArea'].count()/len(housing_data)
    temp_df = temp[temp>0.01].index                                                                         # Get the frequent ones.
    housing_data[feature] = np.where(housing_data[feature].isin(temp_df),housing_data[feature], 'Rare_Var') # Replace the less frequent ones with 'Rare_Var'

In [51]:
housing_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN,YearBuilt_diff,YearRemodAdd_diff,GarageYrBlt_diff
0,1,60,RL,4.174387,9.041922,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,6.752270,854,0,7.444249,1,0,2,1,3,1,Gd,8,Typ,0,Miss_Val,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,12.247694,0,0,0,5,5,5
1,2,20,RL,4.382027,9.169518,Pave,Reg,Lvl,AllPub,FR2,Gtl,Rare_Var,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,7.140453,0,0,7.140453,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,12.109011,0,0,0,31,31,31
2,3,60,RL,4.219508,9.328123,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,6.824374,866,0,7.487734,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,12.317167,0,0,0,7,6,7
3,4,70,RL,4.094345,9.164296,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,6.867974,756,0,7.448334,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,11.849398,0,0,0,91,36,8
4,5,60,RL,4.430817,9.565214,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,7.043160,1053,0,7.695303,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,12.429216,0,0,0,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,4.127134,8.976768,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,6.859615,694,0,7.406711,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,0,8,2007,WD,Normal,12.072541,0,0,0,8,7,8
1456,1457,20,RL,4.442651,9.486076,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,7.636752,0,0,7.636752,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,0,2,2010,WD,Normal,12.254863,0,0,0,32,22,32
1457,1458,70,RL,4.189655,9.109636,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Rare_Var,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,7.080026,1152,0,7.757906,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,2500,5,2010,WD,Normal,12.493130,0,0,0,69,

In [52]:
# Let's encode each categorical value to numerical equivalent for the ease of scaling in the next step
for feature in cat_features:
    labels_ordered = housing_data.groupby([feature])['LotArea'].mean().sort_values().index
    labels_ordered = {k:i for i,k in enumerate(labels_ordered,0)}
    housing_data[feature] = housing_data[feature].map(labels_ordered)

In [53]:
housing_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN,YearBuilt_diff,YearRemodAdd_diff,GarageYrBlt_diff
0,1,60,4,4.174387,9.041922,0,0,0,0,1,0,11,0,0,3,2,7,5,2003,2003,0,0,5,7,1,196.0,2,2,2,3,2,0,4,706,2,0,150,856,1,4,1,3,6.752270,854,0,7.444249,1,0,2,1,3,1,2,8,1,0,0,3,2003,2,2,548,2,3,1,0,61,0,0,0,0,0,2,2008,1,2,12.247694,0,0,0,5,5,5
1,2,20,4,4.382027,9.169518,0,0,0,0,2,0,3,2,0,3,4,6,8,1976,1976,0,0,2,2,0,0.0,1,2,3,3,2,4,3,978,2,0,284,1262,1,4,1,3,7.140453,0,0,7.140453,0,1,2,0,3,1,1,6,1,1,3,3,1976,2,2,460,2,3,1,298,0,0,0,0,0,0,5,2007,1,2,12.109011,0,0,0,31,31,31
2,3,60,4,4.219508,9.328123,0,1,0,0,1,0,11,0,0,3,2,7,5,2001,2002,0,0,5,7,1,162.0,2,2,2,3,2,3,4,486,2,0,434,920,1,4,1,3,6.824374,866,0,7.487734,1,0,2,1,3,1,2,6,1,1,3,3,2001,2,2,608,2,3,1,0,42,0,0,0,0,0,9,2008,1,2,12.317167,0,0,0,7,6,7
3,4,70,4,4.094345,9.164296,0,1,0,0,3,0,18,0,0,3,2,7,5,1915,1970,0,0,7,4,0,0.0,1,2,0,2,4,0,3,216,2,0,540,756,1,3,1,3,6.867974,756,0,7.448334,1,0,1,0,3,1,2,7,1,1,4,1,1998,1,3,642,2,3,1,0,35,272,0,0,0,0,2,2006,1,0,11.849398,0,0,0,91,36,8
4,5,60,4,4.430817,9.565214,0,1,0,0,2,0,20,0,0,3,2,8,5,2000,2000,0,0,5,7,1,350.0,2,2,2,3,2,2,4,655,2,0,490,1145,1,4,1,3,7.043160,1053,0,7.695303,1,0,2,1,4,1,2,9,1,1,3,3,2000,2,3,836,2,3,1,192,84,0,0,0,0,0,12,2008,1,2,12.429216,0,0,0,8,8,8


##### Feature Scaling

There are multiple columns in the dataset with various measurements of different units. 
Scaling the features to common ground aids in better application of ML algorithms

In [54]:
# Let's get the columsn to feature scaling.

scale_features = [feature for feature in housing_data.columns if feature not in ['Id', 'SalePrice']]

In [55]:
# Using MinMaxScaler to scale the values between 0 and 1

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(housing_data[scale_features])

MinMaxScaler()

In [56]:
# Add the scaled data and the ['Id', 'SalePrice'] data to form a complete dataframe
scaled_house_data = pd.concat([housing_data[['Id', 'SalePrice']].reset_index(drop=True),
                                    pd.DataFrame(scaler.transform(housing_data[scale_features]), columns = scale_features)], 
                                    axis = 1 )

In [57]:
scaled_house_data.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN,YearBuilt_diff,YearRemodAdd_diff,GarageYrBlt_diff
0,1,12.247694,0.235294,1.0,0.418208,0.366344,0.0,0.000000,0.0,0.0,0.25,0.0,0.500000,0.0,0.0,0.75,0.4,0.666667,0.500,0.949275,0.883333,0.0,0.0,0.5,0.7,0.25,0.12250,0.666667,0.666667,0.50,0.75,0.5,0.00,0.666667,0.125089,0.333333,0.0,0.064212,0.140098,0.5,1.00,1.0,1.0,0.356155,0.413559,0.0,0.577712,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,0.25,0.000000,0.0,0.6,0.936364,0.666667,0.50,0.386460,0.666667,1.0,0.5,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.333333,0.5,0.0,0.0,0.0,0.036765,0.098361,0.046729
1,2,12.109011,0.000000,1.0,0.495064,0.391317,0.0,0.000000,0.0,0.0,0.50,0.0,0.136364,0.4,0.0,0.75,0.8,0.555556,0.875,0.753623,0.433333,0.0,0.0,0.2,0.2,0.00,0.00000,0.333333,0.666667,0.75,0.75,0.5,1.00,0.500000,0.173281,0.333333,0.0,0.121575,0.206547,0.5,1.00,1.0,1.0,0.503056,0.000000,0.0,0.470245,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,0.25,0.333333,0.6,0.6,0.690909,0.666667,0.50,0.324401,0.666667,1.0,0.5,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.333333,0.5,0.0,0.0,0.0,0.227941,0.524590,0.289720
2,3,12.317167,0.235294,1.0,0.434909,0.422359,0.0,0.333333,0.0,0.0,0.25,0.0,0.500000,0.0,0.0,0.75,0.4,0.666667,0.500,0.934783,0.866667,0.0,0.0,0.5,0.7,0.25,0.10125,0.666667,0.666667,0.50,0.75,0.5,0.75,0.666667,0.086109,0.333333,0.0,0.185788,0.150573,0.5,1.00,1.0,1.0,0.383441,0.419370,0.0,0.593095,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,0.25,0.333333,0.6,0.6,0.918182,0.666667,0.50,0.428773,0.666667,1.0,0.5,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.333333,0.5,0.0,0.0,0.0,0.051471,0.114754,0.065421
3,4,11.849398,0.294118,1.0,0.388581,0.390295,0.0,0.333333,0.0,0.0,0.75,0.0,0.818182,0.0,0.0,0.75,0.4,0.666667,0.500,0.311594,0.333333,0.0,0.0,0.7,0.4,0.00,0.00000,0.333333,0.666667,0.00,0.50,1.0,0.00,0.500000,0.038271,0.333333,0.0,0.231164,0.123732,0.5,0.75,1.0,1.0,0.399941,0.366102,0.0,0.579157,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,0.25,0.333333,0.8,0.2,0.890909,0.333333,0.75,0.452750,0.666667,1.0,0.5,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,0.090909,0.00,0.333333,0.0,0.0,0.0,0.0,0.669118,0.606557,0.074766
4,5,12.429216,0.235294,1.0,0.513123,0.468761,0.0,0.333333,0.0,0.0,0.50,0.0,0.909091,0.0,0.0,0.75,0.4,0.777778,0.500,0.927536,0.833333,0.0,0.0,0.5,0.7,0.25,0.21875,0.666667,0.666667,0.50,0.75,0.5,0.50,0.666667,0.116052,0.333333,0.0,0.209760,0.187398,0.5,1.00,1.0,1.0,0.466237,0.509927,0.0,0.666523,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,0.25,0.333333,0.6,0.6,0.909091,0.666667,0.75,0.589563,0.666667,1.0,0.5,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.333333,0.5,0.0,0.0,0.0,0.058824,0.147541,0.074766


In [58]:
scaled_house_data.to_csv('X_train.csv', index=False)

In [62]:
housing_data = pd.read_csv('test.csv')

# First things first, let's convert the GarageYrBlt to int64
housing_data['GarageYrBlt'] = housing_data['GarageYrBlt'].astype('Int64')

# Let's get the columns with missing values
features_with_na = [feature for feature in housing_data.columns if housing_data[feature].isnull().sum()>1]

# AS we can see, there are 4 features with over 80% of values missing. It's safe to discard them
features_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature']
housing_data = housing_data.drop(features_to_drop, axis=1)
for feature in features_to_drop:
    features_with_na.remove(feature)

# Let's first deal with the missing values in the Categorical columns
cat_na_features = [feature for feature in features_with_na if housing_data[feature].dtype == 'O']

# Let's fill the missing values with 'Miss_val'
def replace_the_missing(housing_data, feature_list):
    house_data = housing_data.copy()
    house_data[feature_list] = house_data[feature_list].fillna('Miss_Val')
    return house_data

housing_data = replace_the_missing(housing_data, cat_na_features)

# Let's now go to the numerical side of the missing values
num_na_features = [feature for feature in features_with_na if feature not in cat_na_features]

for feature in num_na_features:
    # Calculate median
    median_val = housing_data[feature].median()
    
    # Create a new column to denote that there was a missing value (binary substitution) and then replace the current missing value with median
    housing_data[feature+'_NaN'] = np.where(housing_data[feature].isnull(), 1,0)
    housing_data[feature].fillna(median_val, inplace=True)

years_features = [feature for feature in housing_data.columns if ('Yr' in feature or 'Year' in feature) and 'NaN' not in feature]

for feature in years_features:
    if feature != 'YrSold':
        housing_data[feature+'_diff'] = housing_data['YrSold'] - housing_data[feature]

# Let's get the log value to convert the skewed distribution to log normal distribution
skew_features = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']

for feature in skew_features:
    housing_data[feature] = np.log(housing_data[feature])

cat_features = [feature for feature in housing_data.columns if housing_data[feature].dtype == 'O']
for feature in cat_features:
    temp = housing_data.groupby(feature)['LotArea'].count()/len(housing_data)
    temp_df = temp[temp>0.01].index                                                                         # Get the frequent ones.
    housing_data[feature] = np.where(housing_data[feature].isin(temp_df),housing_data[feature], 'Rare_Var') # Replace the less frequent ones with 'Rare_Var'

# Let's encode each categorical value to numerical equivalent for the ease of scaling in the next step
for feature in cat_features:
    labels_ordered = housing_data.groupby([feature])['LotArea'].mean().sort_values().index
    labels_ordered = {k:i for i,k in enumerate(labels_ordered,0)}
    housing_data[feature] = housing_data[feature].map(labels_ordered)

# Let's get the columsn to feature scaling.
scale_features = [feature for feature in housing_data.columns if feature not in ['Id']]

# Using MinMaxScaler to scale the values between 0 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(housing_data[scale_features])

# Add the scaled data and the ['Id', 'SalePrice'] data to form a complete dataframe
scaled_house_data = pd.concat([housing_data[['Id']].reset_index(drop=True),
                                    pd.DataFrame(scaler.transform(housing_data[scale_features]), columns = scale_features)], 
                                    axis = 1 )

scaled_house_data.to_csv('X_test.csv')
scaled_house_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_NaN,MasVnrArea_NaN,BsmtFullBath_NaN,BsmtHalfBath_NaN,GarageYrBlt_NaN,YearBuilt_diff,YearRemodAdd_diff,GarageYrBlt_diff
0,1461,0.000000,1.00,0.593445,0.566360,0.0,0.000000,0.000000,0.0,0.25,0.0,0.40,0.333333,0.0,1.00,0.8,0.444444,0.625,0.625954,0.183333,0.0,0.0,0.8,0.727273,0.0,0.000000,0.000000,0.0,0.50,0.25,0.25,0.0,0.166667,0.116708,0.166667,0.094364,0.126168,0.173111,0.0,0.25,1.0,1.0,0.312253,0.000000,0.0,0.312253,0.0,0.0,0.25,0.0,0.333333,0.5,0.0,0.166667,0.0,0.00,0.0,0.666667,0.211538,0.333333,0.2,0.490591,0.666667,0.666667,0.5,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.000000,0.454545,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.384615,0.822581,0.792994
1,1462,0.000000,0.75,0.598957,0.622527,0.0,0.333333,0.000000,0.0,0.50,0.0,0.40,0.166667,0.0,1.00,0.8,0.555556,0.625,0.603053,0.133333,1.0,0.0,0.7,0.636364,0.5,0.083721,0.000000,0.0,0.50,0.25,0.25,0.0,0.666667,0.230175,0.000000,0.000000,0.189720,0.260844,0.0,0.25,1.0,1.0,0.468253,0.000000,0.0,0.468253,0.0,0.0,0.25,0.5,0.500000,0.5,0.5,0.250000,0.0,0.00,0.0,0.666667,0.201923,0.333333,0.2,0.209677,0.666667,0.666667,0.5,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.735294,0.454545,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.407692,0.870968,0.802548
2,1463,0.235294,0.75,0.558854,0.614005,0.0,0.333333,0.000000,0.0,0.25,0.0,0.70,0.166667,0.0,1.00,0.2,0.444444,0.500,0.900763,0.800000,0.0,0.0,0.8,0.727273,0.0,0.000000,0.000000,0.0,0.75,0.50,0.25,0.0,0.500000,0.197257,0.000000,0.000000,0.064019,0.182139,0.0,0.00,1.0,1.0,0.326139,0.376477,0.0,0.548792,0.0,0.0,0.50,0.5,0.500000,0.5,0.0,0.250000,0.0,0.25,0.6,0.666667,0.326923,1.000000,0.4,0.323925,0.666667,0.666667,0.5,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.107692,0.225806,0.678344
3,1464,0.235294,0.75,0.582212,0.524583,0.0,0.333333,0.000000,0.0,0.25,0.0,0.70,0.166667,0.0,1.00,0.2,0.555556,0.625,0.908397,0.800000,0.0,0.0,0.8,0.727273,0.5,0.015504,0.000000,0.0,0.75,0.25,0.25,0.0,0.500000,0.150125,0.000000,0.000000,0.151402,0.181747,0.0,0.50,1.0,1.0,0.325285,0.364125,0.0,0.542672,0.0,0.0,0.50,0.5,0.500000,0.5,0.5,0.333333,0.0,0.25,0.8,0.666667,0.330128,1.000000,0.4,0.315860,0.666667,0.666667,0.5,0.252809,0.048518,0.0,0.0,0.000000,0.0,0.000000,0.454545,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.225806,0.675159
4,1465,0.588235,0.75,0.317987,0.335596,0.0,0.333333,0.333333,0.0,0.25,0.0,0.55,0.166667,0.0,0.25,0.8,0.777778,0.500,0.862595,0.700000,0.0,0.0,0.5,0.545455,0.0,0.000000,0.666667,0.0,0.75,0.50,0.25,0.0,0.666667,0.065586,0.000000,0.000000,0.475234,0.251227,0.0,0.50,1.0,1.0,0.453388,0.000000,0.0,0.453388,0.0,0.0,0.50,0.0,0.333333,0.5,0.5,0.166667,0.0,0.00,0.0,0.666667,0.310897,0.666667,0.4,0.340054,0.666667,0.666667,0.5,0.000000,0.110512,0.0,0.0,0.250000,0.0,0.000000,0.000000,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.146154,0.322581,0.694268
